In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from collections import Counter
import matplotlib.pyplot as plt

## Popis datasetu 

Využívaná data se týkají přímých marketingových kampaní portugalské banky (Portuguese banking institution). Marketingové kampaně byly realizovány přes telefonní hovory. V několika případech bylo nutné uskutečnit kontakt s klientem vícekrát, pro zjistění, zda došlo k otevření a často bylo třeba provést více než jeden kontakt se stejným klientem, aby bylo možné zjistit, zda byl termínovaný vklad sjednán. 

Zdroj: https://archive.ics.uci.edu/ml/datasets/bank+marketing


In [ ]:
# Load the CSV file into a pandas DataFrame
import pandas as pd

df = pd.read_csv('./data/bank-additional-full.csv', sep=";")

df.head(10)

In [ ]:
print("Rows count: ", df.shape[0])
print("Columns count: ", df.shape[1])

Datový soubor obsahuje 20 proměnných, 
- 10 numerických 
- 10 kategorických 
a jednu cílovou proměnnou "y", která určuje, zda klient sjednal termínovaný vklad a dosahuje hodnoty "yes" (termínovaný vklad sjednán) a "no" (termínovaný vklad nebyl sjednán).

Soubor obsahuje 41188 pozorování. 

Popis proměnných: 
Numerické proměnné
- 1 - Age: Věk klienta v letech.
- 2 - Balance: Zůstatek na účtu klienta (v eurech).
- 3 - Day: Den v měsíci, kdy byl poslední kontakt s klientem.
- 4 - Duration: Délka posledního telefonického kontaktu (v sekundách).
- 5 - Campaign: Počet kontaktů s klientem během aktuální marketingové kampaně.
- 6 - Pdays: Počet dní od posledního kontaktu (999 znamená, že klient byl nikdy předtím nekontaktován).
- 7 - Previous: Počet kontaktů s klientem před aktuální kampaní.
- 8 - Emp.var.rate: Míra zaměstnanecké variability v posledním čtvrtletí (v procentech).
- 9 - Cons.price.idx: Index cen spotřebitelů (v posledním měsíci).
- 10 - Cons.conf.idx: Index důvěry spotřebitelů (v posledním měsíci).

Kategorické proměnné
- 11 - Job: Typ zaměstnání klienta (např. zaměstnanec, důchodce, podnikatel atd.).
- 12 - Marital: Rodinný stav klienta (např. svobodný, ženatý, rozvedený).
- 13 - Education: Vzdělání klienta (např. základní, střední, vysokoškolské).
- 14 - Default: Zda má klient úvěrové selhání (ano/ne).
- 15 - Housing: Zda má klient hypotéku (ano/ne).
- 16 - Loan: Zda má klient osobní půjčku (ano/ne).
- 17 - Contact: Způsob kontaktu (např. mobilní, pevná linka).
- 18 - Month: Měsíc posledního kontaktu (např. leden, únor atd.).
- 19 - Weekday: Den v týdnu posledního kontaktu (např. pondělí, úterý).
- 20 - Poutcome: Výsledek předchozí marketingové kampaně (např. úspěšná, neúspěšná, žádná).

Cílová proměnná 
Subscription: Binární proměnná indikující, zda klient podepsal termínovaný vklad (ano/ne).

In [ ]:
#Data preprocessoring 
df.describe

In [ ]:
df.info()
#zde vidíme, kolik hodnot a jakého typu proměnná je 

In [ ]:
#vizualizace cílové proměnné 
plt.figure(figsize=(6, 4))
df['y'].value_counts().plot(kind='bar', color=['skyblue', 'salmon'])
plt.title('Podepsal klient termínovaný vklad?')
plt.xlabel('y - cílová proměnná  ')
plt.ylabel('Počet klientů')
plt.xticks(rotation=0)
plt.grid(axis='y')

# Zobrazení histogramu
plt.show()

In [ ]:
#vizualizace vstupujících proměnných 
fig = plt.figure(figsize=(20,20))
cols = list(df.columns)
cols.remove("y")

for i, name in enumerate(cols):
    x = fig.add_subplot(5,4,i+1)
    if (df[name].dtype=="object"):
        x.bar(df[name].sort_values().unique(), df[name].value_counts())
    else:
        x.hist(df[name])
    x.set_title(name)

In [ ]:
#chybějící hodnoty - je nutné nahradit hodnoty "unknown" hodnotami NA 

df2=df.replace(to_replace="unknown",value=pd.NA)
df2["pdays"]=df2[["pdays"]].replace(to_replace=999,value=np.nan)
df2.head()


In [ ]:
#nyní můžeme zjistit chybějící hodnoty a jejich počet
na=df2.isnull()
na.sum()

In [ ]:
#korelace 
df_encoded = pd.get_dummies(df, drop_first=True)

# Vytvoření korelační matice
correlation_matrix = df_encoded.corr()

#vizualizace 
plt.figure(figsize=(30, 30))
sns.heatmap(correlation_matrix, annot=True, fmt=".2f", cmap='coolwarm', square=True)
plt.title('Korelační matice')
plt.show()

Na základě informací o datovém souboru a chybějících proměnných jsme se rozhodli z analýzy vyřadit "contact", protože pro výsledek není relevantiní zda byl kontaktován přes pevnou linku nebo mobilní telefon. 
Pro velké množství chybějících pozorování odstraníme také proměnnou "poutcome". 
Pro analýzu je nutné také odstranit proměnnou "nr.employed", protože hodnota je získána až po realizaci telefonního hovoru a ovlivňuje cílovou proměnnou. Pokud dosahuje nr.employed hodnoty 0, pak cílová proměnná = "no". 

In [12]:
df2=df2.drop(["pdays", "duration", "nr.employed","poutcome","contact"],axis=1)
#df2.head(10)

Dataset neobsahuje také úplné informace o čase, pozorování jsou však sesbírána chronologicky. Vytvoříme novou proměnnou "year".  

In [ ]:
df2["year"] = 2008
add_year = 0
months = ["jan", "feb", "mar", "apr", "may", "jun", "jul", "aug", "sep", "oct", "nov", "dec"]
actual_month = df2.loc[0, "month"]  

for i in df2.index:
    if (months.index(actual_month) > months.index(df2.loc[i, "month"])):
        add_year += 1  
    df2.loc[i, "year"] += add_year
    actual_month = df2.loc[i, "month"]  

print(df2.head())

In [ ]:
#vytvoření nových intervalů pro proměnné "previous" a "campaign"

fig = plt.figure(figsize=(20,10))
ax1 = fig.add_subplot(1, 2, 1)
ax2 = fig.add_subplot(1, 2, 2)

ax1.bar(df["previous"].sort_values().unique(), df["previous"].value_counts())
ax1.set_title('Boxplot of attribute "previous"')
ax2.bar(df["campaign"].sort_values().unique(), df["campaign"].value_counts())
ax2.set_title('Boxplot of attribute "campaign"')

fig.show

In [15]:
df2["previous_cat"] = pd.cut(df2.previous, bins=[-1, 0, 2, 1000], labels=["0", "1-2", "3+"])
df2["campaign_cat"] = pd.cut(df2.campaign, bins=[-1, 0, 1, 2, 3, 4, 10, 1000], labels=["0", "1", "2", "3", "4", "5-10", "10+"])
#df2.head
# Zobrazení prvních několika řádků s novými kategoriemi
#print(df2[["previous", "previous_cat", "campaign", "campaign_cat"]].head())


In [ ]:
#řešení missing values - nahrazení NaN hodnot nejčastějšími hodnotami (mód)

for col in df2.columns:
    if df2[col].dtype == "object" or df2[col].dtype == "category":
        mode_value = df2[col].mode()[0]  # Získání módu
        df2[col].fillna(mode_value, inplace=True) 
        

#print(df2)

In [ ]:

# převod proměnných typu object nebo category na číselné hodnoty 0 = no, 1 = yes a kategorizace  
#kodovník 

data = pd.DataFrame()

# Slovník pro uložení kódovníků
kodovniky = {}

for col in df2.columns:
    if df2[col].dtype == "object" or df2[col].dtype == "category":
        # Faktorizace sloupce
        codes, uniques = pd.factorize(df2[col])
        data[col] = codes + 1  # Přiřadíme 1, 2, 3 místo 0, 1, 2
        
        # Uložíme kódovník
        kodovniky[col] = dict(enumerate(uniques, 1))  # +1 pro lepší čitelnost (začínáme od 1)
    else:
        data[col] = df2[col]

# Úprava sloupce 'y'
data['y'] = data['y'] - 1

# Výpis všech kódovníků
for col, kodovnik in kodovniky.items():
    print(f"Kódovník pro sloupec '{col}': {kodovnik}")

In [719]:
#kontrola datasetu 
#data.head(10)


In [720]:
#data.describe

In [721]:
#data.info 

In [722]:
#na2=data.isnull()
#na2.sum()

In [723]:
# Oddělení atributů (features) a cílové proměnné (target)
X = df.drop(columns=['y'])  # Všechny sloupce kromě 'y'
y = df['y']  # Cílová proměnná


In [ ]:
# Rozdělené dat na trénovací a testovací

# Oddělení atributů (features) a cílové proměnné (target)
X = df2.drop(columns=['y'])  # Všechny sloupce kromě 'y' - cílové proměnné
y = df2['y']  # Cílová proměnná

# Rozdělení dat na trénovací a testovací sadu
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) # 20% testovací, 80% trénovací. Random state znamená, že při každém stuštění bude stejné rozložení dat

# Výpis velikosti trénovací a testovací sady
print("Počet záznamů v trénovací sadě:", X_train.shape[0])
print("Počet záznamů v testovací sadě:", X_test.shape[0])